In [67]:
import requests
from bs4 import BeautifulSoup
import json
from lxml import etree, html as lhtml

In [3]:
relod_url = 'https://shop.relod.ru/'

In [4]:
fiction_url = f'{relod_url}catalog-products/4577/?num=20&sort=PROPERTY_RATING&order=desc'

In [64]:
data = {"siteId": "s1",
        "template": "bxready.73c1485ff4ce40486bbd30b6d9f2f4df8fafe937859e42fbb8f388a4a0518226",
        "parameters": "YToxMTk6e3M6MTg6IkNPTVBPTkVOVF9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxMToiSUJMT0NLX1RZUEUiO3M6NzoiY2F0YWxvZyI7czo5OiJJQkxPQ0tfSUQiO3M6MToiMiI7czoxMDoiU0VDVElPTl9JRCI7czo0OiI0NTc3IjtzOjEyOiJTRUNUSU9OX0NPREUiO047czoxMToiRklMVEVSX05BTUUiO3M6MTY6IkNBVEFMT0dfUFJPRFVDVFMiO3M6MTk6IklOQ0xVREVfU1VCU0VDVElPTlMiO3M6MToiQSI7czoxOToiU0hPV19BTExfV09fU0VDVElPTiI7czoxOiJOIjtzOjEzOiJDVVNUT01fRklMVEVSIjtzOjA6IiI7czoxODoiSElERV9OT1RfQVZBSUxBQkxFIjtzOjE6Ik4iO3M6MjU6IkhJREVfTk9UX0FWQUlMQUJMRV9PRkZFUlMiO3M6MToiWSI7czoxODoiRUxFTUVOVF9TT1JUX0ZJRUxEIjtzOjE1OiJQUk9QRVJUWV9SQVRJTkciO3M6MTg6IkVMRU1FTlRfU09SVF9PUkRFUiI7czo0OiJkZXNjIjtzOjE5OiJFTEVNRU5UX1NPUlRfRklFTEQyIjtzOjQ6Im5hbWUiO3M6MTk6IkVMRU1FTlRfU09SVF9PUkRFUjIiO3M6NDoiZGVzYyI7czoxNzoiT0ZGRVJTX1NPUlRfRklFTEQiO3M6MjoiaWQiO3M6MTc6Ik9GRkVSU19TT1JUX09SREVSIjtzOjM6ImFzYyI7czoxODoiT0ZGRVJTX1NPUlRfRklFTEQyIjtzOjU6InNob3dzIjtzOjE4OiJPRkZFUlNfU09SVF9PUkRFUjIiO3M6MzoiYXNjIjtzOjE4OiJQQUdFX0VMRU1FTlRfQ09VTlQiO2k6MjA7czoxODoiTElORV9FTEVNRU5UX0NPVU5UIjtpOjIwO3M6MTM6IlBST1BFUlRZX0NPREUiO2E6MTc6e2k6MDtzOjg6IkFUUl9JU0JOIjtpOjE7czoxNjoiQVRSX1NUT0NLX1NUQVRVUyI7aToyO3M6MTM6IkFUUl9QVUJMSVNIRVIiO2k6MztzOjEwOiJBVFJfQVVUSE9SIjtpOjQ7czoxMjoiQVRSX0xBTkdVQUdFIjtpOjU7czoxMDoiQVRSX1NFUklFUyI7aTo2O3M6MTQ6IkFUUl9TVUJfU0VSSUVTIjtpOjc7czoxMToiQVRSX0VESVRJT04iO2k6ODtzOjc6IkFUUl9BR0UiO2k6OTtzOjEwOiJBVFJfRk9STUFUIjtpOjEwO3M6MTE6IkFUUl9DQVJSSUVSIjtpOjExO3M6MTY6IkFUUl9QUk9EVUNUX1RZUEUiO2k6MTI7czo5OiJBVFJfUEFHRVMiO2k6MTM7czoxMjoiQVRSX1BVQl9EQVRFIjtpOjE0O3M6MTA6IkFUUl9PUklHSU4iO2k6MTU7czo2OiJyYXRpbmciO2k6MTY7czowOiIiO31zOjIwOiJQUk9QRVJUWV9DT0RFX01PQklMRSI7YToyOntpOjA7czo4OiJBVFJfSVNCTiI7aToxO3M6MTM6IkFUUl9QVUJMSVNIRVIiO31zOjE3OiJPRkZFUlNfRklFTERfQ09ERSI7YTo2OntpOjA7czo0OiJOQU1FIjtpOjE7czoxMjoiUFJFVklFV19URVhUIjtpOjI7czoxNToiUFJFVklFV19QSUNUVVJFIjtpOjM7czoxMToiREVUQUlMX1RFWFQiO2k6NDtzOjE0OiJERVRBSUxfUElDVFVSRSI7aTo1O3M6MDoiIjt9czoyMDoiT0ZGRVJTX1BST1BFUlRZX0NPREUiO2E6Mjp7aTowO3M6NjoiTk9USUNFIjtpOjE7czowOiIiO31zOjEyOiJPRkZFUlNfTElNSVQiO3M6MToiMCI7czoyMDoiUFJPRFVDVF9ESVNQTEFZX01PREUiO3M6MToiWSI7czoxMzoiQUREX1BJQ1RfUFJPUCI7czoxNzoiQVRSX0VYVFJBX1BJQ1RVUkUiO3M6MTk6Ik9GRkVSX0FERF9QSUNUX1BST1AiO3M6MToiLSI7czoxNjoiT0ZGRVJfVFJFRV9QUk9QUyI7YToxOntpOjA7czo2OiJOT1RJQ0UiO31zOjIwOiJQUk9EVUNUX1NVQlNDUklQVElPTiI7czoxOiJZIjtzOjIxOiJTSE9XX0RJU0NPVU5UX1BFUkNFTlQiO3M6MToiWSI7czoxNDoiU0hPV19PTERfUFJJQ0UiO3M6MToiWSI7czoxNzoiU0hPV19NQVhfUVVBTlRJVFkiO3M6MToiQSI7czoyMToiQlhSX1NIT1dfTUFYX1FVQU5USVRZIjtzOjE6IkEiO3M6MTY6IlNIT1dfQ0xPU0VfUE9QVVAiO3M6MToiWSI7czoxMjoiTUVTU19CVE5fQlVZIjtzOjEyOiLQmtGD0L/QuNGC0YwiO3M6MjI6Ik1FU1NfQlROX0FERF9UT19CQVNLRVQiO3M6MTc6ItCSINC60L7RgNC30LjQvdGDIjtzOjE4OiJNRVNTX0JUTl9TVUJTQ1JJQkUiO3M6NDQ6ItCj0LLQtdC00L7QvNC40YLRjCDQviDQv9C+0YHRgtGD0L/Qu9C10L3QuNC4IjtzOjE1OiJNRVNTX0JUTl9ERVRBSUwiO3M6MzQ6ItCf0L7QtNGA0L7QsdC90LXQtSDQviDRgtC+0LLQsNGA0LUiO3M6MTg6Ik1FU1NfTk9UX0FWQUlMQUJMRSI7czoyNDoi0J3QtdGCINCyINC90LDQu9C40YfQuNC4IjtzOjExOiJTRUNUSU9OX1VSTCI7czozMToiL2NhdGFsb2ctcHJvZHVjdHMvI1NFQ1RJT05fSUQjLyI7czoxMDoiREVUQUlMX1VSTCI7czozMzoiL2NhdGFsb2ctcHJvZHVjdHMvI0VMRU1FTlRfQ09ERSMvIjtzOjE5OiJTRUNUSU9OX0lEX1ZBUklBQkxFIjtzOjEwOiJTRUNUSU9OX0lEIjtzOjEwOiJDQUNIRV9UWVBFIjtzOjE6IkEiO3M6MTA6IkNBQ0hFX1RJTUUiO3M6ODoiMzYwMDAwMDAiO3M6MTI6IkNBQ0hFX0ZJTFRFUiI7czoxOiJZIjtzOjEyOiJDQUNIRV9HUk9VUFMiO3M6MToiWSI7czo5OiJTRVRfVElUTEUiO3M6MToiWSI7czoxMzoiQlJPV1NFUl9USVRMRSI7czoxOiItIjtzOjEzOiJNRVRBX0tFWVdPUkRTIjtzOjE6Ii0iO3M6MTY6Ik1FVEFfREVTQ1JJUFRJT04iO3M6MToiLSI7czoxNzoiU0VUX0xBU1RfTU9ESUZJRUQiO3M6MToiWSI7czoyNDoiVVNFX01BSU5fRUxFTUVOVF9TRUNUSU9OIjtzOjE6IlkiO3M6MTg6IkFERF9TRUNUSU9OU19DSEFJTiI7czoxOiJOIjtzOjE1OiJBQ1RJT05fVkFSSUFCTEUiO3M6NjoiYWN0aW9uIjtzOjE5OiJQUk9EVUNUX0lEX1ZBUklBQkxFIjtzOjI6ImlkIjtzOjEwOiJQUklDRV9DT0RFIjthOjE6e2k6MDtzOjEwOiJCQVNFX1BSSUNFIjt9czoxNToiVVNFX1BSSUNFX0NPVU5UIjtzOjE6Ik4iO3M6MTY6IlNIT1dfUFJJQ0VfQ09VTlQiO3M6MDoiIjtzOjE3OiJQUklDRV9WQVRfSU5DTFVERSI7czoxOiJZIjtzOjE2OiJDT05WRVJUX0NVUlJFTkNZIjtzOjE6IlkiO3M6MTE6IkNVUlJFTkNZX0lEIjtzOjM6IlJVQiI7czoxMDoiQkFTS0VUX1VSTCI7czoyMDoiL3BlcnNvbmFsL2Jhc2tldC5waHAiO3M6MjA6IlVTRV9QUk9EVUNUX1FVQU5USVRZIjtzOjE6IlkiO3M6MjU6IlBST0RVQ1RfUVVBTlRJVFlfVkFSSUFCTEUiO3M6ODoicXVhbnRpdHkiO3M6MjQ6IkFERF9QUk9QRVJUSUVTX1RPX0JBU0tFVCI7czoxOiJZIjtzOjIyOiJQUk9EVUNUX1BST1BTX1ZBUklBQkxFIjtzOjQ6InByb3AiO3M6MjY6IlBBUlRJQUxfUFJPRFVDVF9QUk9QRVJUSUVTIjtzOjE6IlkiO3M6MTg6IlBST0RVQ1RfUFJPUEVSVElFUyI7YTo0OntpOjA7czoxMDoiQVRSX0FVVEhPUiI7aToxO3M6MTI6IkFUUl9MQU5HVUFHRSI7aToyO3M6NzoiQVRSX0FHRSI7aTozO3M6MTA6IkFUUl9PUklHSU4iO31zOjIyOiJPRkZFUlNfQ0FSVF9QUk9QRVJUSUVTIjthOjE6e2k6MDtzOjY6Ik5PVElDRSI7fXM6MjA6IkFERF9UT19CQVNLRVRfQUNUSU9OIjtOO3M6MTU6IkRJU1BMQVlfQ09NUEFSRSI7czoxOiJOIjtzOjE0OiJQQUdFUl9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxNzoiRElTUExBWV9UT1BfUEFHRVIiO3M6MToiWSI7czoyMDoiRElTUExBWV9CT1RUT01fUEFHRVIiO3M6MToiWSI7czoxMToiUEFHRVJfVElUTEUiO3M6MTI6ItCi0L7QstCw0YDRiyI7czoxNzoiUEFHRVJfU0hPV19BTFdBWVMiO3M6MToiTiI7czoyMDoiUEFHRVJfREVTQ19OVU1CRVJJTkciO3M6MToiTiI7czozMToiUEFHRVJfREVTQ19OVU1CRVJJTkdfQ0FDSEVfVElNRSI7czo1OiIzNjAwMCI7czoxNDoiUEFHRVJfU0hPV19BTEwiO3M6MToiTiI7czoyMjoiUEFHRVJfQkFTRV9MSU5LX0VOQUJMRSI7czoxOiJOIjtzOjE1OiJQQUdFUl9CQVNFX0xJTksiO047czoxNzoiUEFHRVJfUEFSQU1TX05BTUUiO047czo5OiJMQVpZX0xPQUQiO3M6MToiWSI7czoxODoiTUVTU19CVE5fTEFaWV9MT0FEIjtzOjIzOiLQn9C+0LrQsNC30LDRgtGMINC10YnRkSI7czoxNDoiTE9BRF9PTl9TQ1JPTEwiO3M6MToiTiI7czoxNDoiU0VUX1NUQVRVU180MDQiO3M6MToiWSI7czo4OiJTSE9XXzQwNCI7czoxOiJZIjtzOjg6IkZJTEVfNDA0IjtzOjg6Ii80MDQucGhwIjtzOjExOiJNRVNTQUdFXzQwNCI7czowOiIiO3M6MTU6IkNPTVBBVElCTEVfTU9ERSI7czoxOiJZIjtzOjI4OiJESVNBQkxFX0lOSVRfSlNfSU5fQ09NUE9ORU5UIjtzOjE6Ik4iO3M6MjI6IlVTRV9FTkhBTkNFRF9FQ09NTUVSQ0UiO3M6MToiWSI7czoxMjoiRU5MQVJHRV9QUk9QIjtzOjA6IiI7czoxMjoiQ09NUEFSRV9QQVRIIjtzOjUwOiIvY2F0YWxvZy1wcm9kdWN0cy9jb21wYXJlLnBocD9hY3Rpb249I0FDVElPTl9DT0RFIyI7czoxNjoiTUVTU19CVE5fQ09NUEFSRSI7czoxNjoi0KHRgNCw0LLQvdC40YLRjCI7czoxMjoiQ09NUEFSRV9OQU1FIjtzOjIwOiJDQVRBTE9HX0NPTVBBUkVfTElTVCI7czoyMjoiTUVTU19TSE9XX01BWF9RVUFOVElUWSI7czoxNDoi0J3QsNC70LjRh9C40LUiO3M6MjQ6IlJFTEFUSVZFX1FVQU5USVRZX0ZBQ1RPUiI7czoyOiIxMCI7czoyNzoiTUVTU19SRUxBVElWRV9RVUFOVElUWV9NQU5ZIjtzOjEwOiLQvNC90L7Qs9C+IjtzOjI2OiJNRVNTX1JFTEFUSVZFX1FVQU5USVRZX0ZFVyI7czo4OiLQvNCw0LvQviI7czoxNzoiUVVBTlRJVFlfSU5fU1RPQ0siO3M6MTc6ItCSINC90LDQu9C40YfQuNC4IjtzOjE4OiJRVUFOVElUWV9PVVRfU1RPQ0siO3M6MTc6ItCf0L7QtCDQt9Cw0LrQsNC3IjtzOjI0OiJISURFX1NFQ1RJT05fREVTQ1JJUFRJT04iO3M6MToiWSI7czoxMjoiVEhJU19VTklDX0lEIjtzOjk6IjFfc2VjdGlvbiI7czo2OiJSRUdJT04iO3M6MDoiIjtzOjIwOiJCWFJfQUpBWF9SRUdJT05fSU5GTyI7czowOiIiO3M6MzM6IkJYUkVBRFlfRUxFTUVOVF9BRERDTEFTU19MSVNUUEFHRSI7czowOiIiO3M6Mjk6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX0xJU1RQQUdFIjtzOjIxOiJlY29tbWVyY2UubTIudGFibGUudjEiO3M6MzU6IkJYUkVBRFlfRUxFTUVOVF9FWFRfUEFSQU1TX0xJU1RQQUdFIjtzOjEyOiJhcnJFeHRQYXJhbXMiO3M6Mjc6IkJYUkVBRFlfVVNFUl9UWVBFU19MSVNUUEFHRSI7czoxOiJOIjtzOjMzOiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfTElTVFBBR0UiO3M6MToiWSI7czozMToiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9MSVNUUEFHRSI7czoxOiJZIjtzOjE3OiJDVVJSRU5UX0JBU0VfUEFHRSI7czo3MzoiL2NhdGFsb2ctcHJvZHVjdHMvNDU3Ny8/dmlldz10YWJsZSZudW09MjAmc29ydD1QUk9QRVJUWV9SQVRJTkcmb3JkZXI9ZGVzYyI7czoxMToiUEFSRU5UX05BTUUiO3M6MjM6ImJ4cmVhZHkubWFya2V0MjpjYXRhbG9nIjtzOjIwOiJQQVJFTlRfVEVNUExBVEVfTkFNRSI7czoxMzoicmVsb2QtY2F0YWxvZyI7czoyMDoiUEFSRU5UX1RFTVBMQVRFX1BBR0UiO3M6Nzoic2VjdGlvbiI7czoxMzoiR0xPQkFMX0ZJTFRFUiI7YTowOnt9fQ==.00631674e61b33c8e8aa3994ce36c904c9993f986d212be47e77fd4ca26223d4",
        "action": "showMore",
        "PAGEN_1": "0"
}

In [65]:
for i in range(250):
    pass

In [66]:
r = requests.post('https://shop.relod.ru/bitrix/components/bxready.market2/catalog.section/ajax.php', data=data)

In [69]:
tree = lhtml.fromstring(r.content)

In [78]:
tree

<Element div at 0x2af4b6c3ef0>

In [147]:
table = tree.xpath('//div[@data-entity="container-1"]/div//div[starts-with(@class, "col")]')

In [148]:
table

[]

In [129]:
dir(tree)

['__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_init',
 'addnext',
 'addprevious',
 'append',
 'attrib',
 'base',
 'base_url',
 'body',
 'classes',
 'clear',
 'cssselect',
 'drop_tag',
 'drop_tree',
 'extend',
 'find',
 'find_class',
 'find_rel_links',
 'findall',
 'findtext',
 'forms',
 'get',
 'get_element_by_id',
 'getchildren',
 'getiterator',
 'getnext',
 'getparent',
 'getprevious',
 'getroottree',
 'head',
 'index',
 'insert',
 'items',
 'iter',
 'iterancestors',
 'iterchildren',
 'iterdescendants',
 'iterfind',


In [128]:
tree.

TypeError: 'NoneType' object is not callable

In [114]:
divs = tree.xpath('//a[@class="bxr-item-image-wrap")]//atribute::href')

XPathEvalError: Invalid predicate

In [109]:
divs

[<Element a at 0x2af4bacdf90>,
 <Element a at 0x2af4bacd540>,
 <Element a at 0x2af4baa1a90>,
 <Element a at 0x2af4baa1ae0>,
 <Element a at 0x2af4baa1b30>,
 <Element a at 0x2af4baa1b80>,
 <Element a at 0x2af4baa1bd0>,
 <Element a at 0x2af4baa1c20>,
 <Element a at 0x2af4baa1c70>,
 <Element a at 0x2af4baa1cc0>,
 <Element a at 0x2af4baa1d10>,
 <Element a at 0x2af4baa1d60>,
 <Element a at 0x2af4baa1db0>,
 <Element a at 0x2af4baa1e00>,
 <Element a at 0x2af4baa1e50>,
 <Element a at 0x2af4baa1ea0>,
 <Element a at 0x2af4baa1ef0>,
 <Element a at 0x2af4baa1f40>,
 <Element a at 0x2af4baa1f90>,
 <Element a at 0x2af4b0ec040>,
 <Element a at 0x2af4b0ec090>,
 <Element a at 0x2af4b0ec0e0>,
 <Element a at 0x2af4b0ec130>,
 <Element a at 0x2af4b0ec180>,
 <Element a at 0x2af4b0ec1d0>,
 <Element a at 0x2af4b0ec220>,
 <Element a at 0x2af4b0ec270>,
 <Element a at 0x2af4b0ec2c0>,
 <Element a at 0x2af4b0ec310>,
 <Element a at 0x2af4b0ec360>,
 <Element a at 0x2af4b0ec3b0>,
 <Element a at 0x2af4b0ec400>,
 <Elemen

In [60]:
table = fiction_soup.find('div', {'class': 'row'})

In [63]:
table.find_all_next('div')

[<div class="t_1_section col-xl-12 col-lg-12 col-md-12 col-sm-12 col-xs-12" data-entity="items-row">
 <div class="bxr-ecommerce-m2-table-v1" data-resize="1" data-uid="1_section" id="bx_1877656419_267466_e0543db0da0d6c005d17ab7bc29d948f">
 <table class="bxr-element-container">
 <tr>
 <td class="bxr-element-image-col hidden-sm hidden-xs">
 <div class="bxr-element-image">
 <a href="/catalog-products/the-pillars-of-the-earth/">
 <img alt="The Pillars of the Earth" id="bx_1877656419_267466_e0543db0da0d6c005d17ab7bc29d948f_pict" src="//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/resize_cache/iblock/41a/50_50_1/9781509886067.jpg?15806200562247" title="The Pillars of the Earth"/>
 </a>
 </div> </td>
 <td class="bxr-element-name-col hidden-sm hidden-xs">
 <div class="bxr-element-name" id="bxr-element-name-267466">
 <a href="/catalog-products/the-pillars-of-the-earth/">The Pillars of the Earth</a>
 </div> <div class="bxr-element-avail" id="bx_1877656419_267466_e0543db0da0d6c005d17ab7bc29d948f_avail"

In [59]:
divs

[<div class="t_1_section col-xl-12 col-lg-12 col-md-12 col-sm-12 col-xs-12" data-entity="items-row">
 <div class="bxr-ecommerce-m2-table-v1" data-resize="1" data-uid="1_section" id="bx_1877656419_267466_e0543db0da0d6c005d17ab7bc29d948f">
 <table class="bxr-element-container">
 <tr>
 <td class="bxr-element-image-col hidden-sm hidden-xs">
 <div class="bxr-element-image">
 <a href="/catalog-products/the-pillars-of-the-earth/">
 <img alt="The Pillars of the Earth" id="bx_1877656419_267466_e0543db0da0d6c005d17ab7bc29d948f_pict" src="//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/resize_cache/iblock/41a/50_50_1/9781509886067.jpg?15806200562247" title="The Pillars of the Earth"/>
 </a>
 </div> </td>
 <td class="bxr-element-name-col hidden-sm hidden-xs">
 <div class="bxr-element-name" id="bxr-element-name-267466">
 <a href="/catalog-products/the-pillars-of-the-earth/">The Pillars of the Earth</a>
 </div> <div class="bxr-element-avail" id="bx_1877656419_267466_e0543db0da0d6c005d17ab7bc29d948f_avail"

In [57]:
divs[-1].find('a')

In [51]:
links = []
for div in divs:
    links.append(div.find('a')['href'])

TypeError: 'NoneType' object is not subscriptable

In [29]:
links = {a['href'] for a in divs}

In [32]:
links

{'/catalog-products/bleach-vol-74/',
 '/catalog-products/eleanor_oliphant_is_completely_fine_debut_bestseller_and_costa_first_novel_book_award_winner_2017/',
 '/catalog-products/tess_of_the_d_urbervilles/',
 '/catalog-products/the-pillars-of-the-earth/',
 '/catalog-products/the_casual_vacancy_9780751552867/',
 '/catalog-products/the_essential_tales_of_chekhov/',
 '/catalog-products/the_fellowship_of_the_ring/',
 '/catalog-products/the_names/',
 '/catalog-products/tokyo-ghoul-re-vol-9/',
 '/catalog-products/zombie_loan_v_1/',
 '/catalog-products/zombie_loan_v_10/',
 '/catalog-products/zombie_loan_v_3/',
 '/catalog-products/zoo_city/',
 '/catalog-products/zoo_tails/',
 '/catalog-products/zoo_the_graphic_novel/',
 '/catalog-products/zorro/',
 '/catalog-products/zoya/',
 '/catalog-products/zuckerman_unbound/',
 '/catalog-products/zuleikha/',
 '/catalog-products/zulu_hart_9780340953648/',
 '/info/how-to-sroki/'}

In [73]:
r.content

b'        <div class="bxr-top-pager" data-pagination-num="1">\n            <!-- pagination-container -->\n            \n<div class="navigation">\n\t\t<div class="navigation-pages">\n\t\t\t<span class="navigation-button navigation-disabled"></span>\t\t\t\t\t\t\t<span class="nav-current-page first-current-page bxr-font-color bxr-border-color">1</span>\n\t\t\t\t\t\t\t\t\t<a href="/catalog-products/4577/?view=table&num=20&sort=PROPERTY_RATING&order=desc&amp;PAGEN_1=2" class="bxr-font-color">2</a>\n\t\t\t\t<span class="navigation-page-other">...</span>\t\t\t\t\t<a href="/catalog-products/4577/?view=table&num=20&sort=PROPERTY_RATING&order=desc&amp;PAGEN_1=595" class="bxr-font-color">595</a>\n\t\t\t\t\t\t\t<a href="/catalog-products/4577/?view=table&num=20&sort=PROPERTY_RATING&order=desc&amp;PAGEN_1=2" id="navigation_1_next_page" class="navigation-button">                <i class="fa fa-angle-right" aria-hidden="true"></i>\n            </a>\t\t\t</div> <!-- //navigation-pages-->\n</div>\n<scr

In [69]:
j = json.loads(r.content)

JSONDecodeError: Expecting value: line 1 column 9 (char 8)